In [1]:
from transformers import pipeline

In [2]:
## 检查当前工作目录和文件
import os
print("当前工作目录:", os.getcwd())
print("目录下有哪些文件:", os.listdir(os.getcwd()))

当前工作目录: C:\Users\Lenovo\Desktop
目录下有哪些文件: ['.gradio', '.ipynb_checkpoints', '2025校历.pdf', '2026校历.pdf', '2503.05320v4.pdf', 'Anaconda', 'Anaconda Navigator.lnk', 'Ass封面 复制使用.docx', 'Bashes', 'BigData(1)(1).docx', 'BigData(ChaChongBan).docx', 'Code.exe - 快捷方式.lnk', 'CodeBlocks.lnk', 'Comet.lnk', 'ContextMenuManager.NET.4.0.exe - 快捷方式.lnk', 'desktop.ini', 'Dism++x86.exe - 快捷方式.lnk', 'Docker Desktop.lnk', 'Eclipse IDE for Enterprise Java and Web Developers - 2024-03.lnk', 'eclipse_File - 快捷方式.lnk', 'FYP2', 'FYP课', 'Git Bash.lnk', 'GitHub Desktop.lnk', 'Goose Goose Duck.url', 'Grammarly.lnk', 'Jupyter Notebook.lnk', 'Jupyter_Server_8x8.bat', 'LaTex', 'Legion Zone.lnk', 'MiKTeX Console.lnk', 'MySQL Shell.lnk', 'MySQL Workbench 8.0 CE.lnk', 'Node.js.lnk', 'PE3', 'PythonFile', 'python_works - 快捷方式.lnk', 'SpringToolSuite4.exe - 快捷方式.lnk', 'SQLiteStudio.exe - 快捷方式.lnk', 'Stardew Valley.url', 'Start Tor Browser.lnk', 'studio64.exe - 快捷方式.lnk', 'TeXworks.lnk', 'WorkSpace4vscode - 快捷方式.lnk', 'Xmin

In [3]:
# 1. 指定包含你截图文件的文件夹路径
# 如果文件在当前目录的 'final_model' 文件夹下：
local_model_path = "D:\\WorkSpace4vscode\\FYP\\distilbert_finedtuned_final" 

# 2. 加载 Pipeline
# 这里的 "text-classification" 假设你做的是文本分类任务
# 如果你做的是其他任务（如 token-classification），请相应修改
classifier = pipeline("text-classification", model=local_model_path, tokenizer=local_model_path)



Device set to use cpu


In [4]:
# 3. 进行测试
# Testing with a sample input
text_input = " This is a sample text for classification."

result = classifier(text_input)

print(f"Input: {text_input}")
print(f"Prediction: {result}")

Input:  This is a sample text for classification.
Prediction: [{'label': 'LABEL_0', 'score': 0.9987421631813049}]


In [5]:
import gradio as gr

# Define prediction function
def detect_sql_injection(text):
    """Detect SQL injection in input text"""
    if not text.strip():
        return "⚠️ Please enter text for detection", "", ""
    
    result = classifier(text)
    label = result[0]['label']
    score = result[0]['score']
    
    # Determine risk level and color
    if label.upper() in ['SQLI', 'SQL_INJECTION', 'LABEL_1', '1']:
        risk = "🚨 HIGH RISK - SQL Injection Detected"
        color = "red"
        suggestion = "⚠️ This input contains potential SQL injection patterns. Do not execute this query!"
    else:
        risk = "✅ SAFE - No SQL Injection Detected"
        color = "green"
        suggestion = "✓ This input appears to be safe."
    
    confidence = f"Confidence: {score:.2%}"
    
    return risk, confidence, suggestion

# Login simulation
def check_login(username, password):
    """Simulate login and check for SQL injection"""
    combined_input = f"{username} {password}"
    risk, confidence, suggestion = detect_sql_injection(combined_input)
    
    login_info = f"**Username:** `{username}`\n**Password:** `{'*' * len(password)}`"
    return login_info, risk, confidence, suggestion

# Comment simulation
def check_comment(comment):
    """Simulate comment submission and check for SQL injection"""
    return detect_sql_injection(comment)

# Create Gradio Blocks interface for more advanced layout
with gr.Blocks(theme=gr.themes.Soft(), title="SQL Injection Detection System") as demo:
    
    gr.Markdown("""
    # 🛡️ SQL Injection Detection System
    ### Powered by DistilBERT Deep Learning Model
    Test various scenarios to detect SQL injection attempts in real-time.
    """)
    
    with gr.Tabs():
        # Tab 1: Login Scenario
        with gr.Tab("🔐 Login Scenario"):
            gr.Markdown("### Simulate a login form and detect SQL injection attempts")
            
            with gr.Row():
                with gr.Column():
                    login_username = gr.Textbox(
                        label="Username",
                        placeholder="Enter username...",
                        lines=1
                    )
                    login_password = gr.Textbox(
                        label="Password",
                        placeholder="Enter password...",
                        type="password",
                        lines=1
                    )
                    login_btn = gr.Button("🔍 Check Login", variant="primary")
                    
                    gr.Markdown("#### Try these examples:")
                    gr.Examples(
                        examples=[
                            ["admin", "password123"],
                            ["admin' OR '1'='1", "anything"],
                            ["admin'--", ""],
                            ["' OR 1=1--", "password"],
                        ],
                        inputs=[login_username, login_password],
                        label="Example Inputs"
                    )
                
                with gr.Column():
                    login_info = gr.Markdown("**Input will be displayed here**")
                    login_risk = gr.Textbox(label="Risk Assessment", lines=1)
                    login_confidence = gr.Textbox(label="Model Confidence", lines=1)
                    login_suggestion = gr.Textbox(label="Security Suggestion", lines=2)
            
            login_btn.click(
                fn=check_login,
                inputs=[login_username, login_password],
                outputs=[login_info, login_risk, login_confidence, login_suggestion]
            )
        
        # Tab 2: Comment Scenario
        with gr.Tab("💬 Comment Scenario"):
            gr.Markdown("### Simulate a comment submission and detect SQL injection")
            
            with gr.Row():
                with gr.Column():
                    comment_input = gr.Textbox(
                        label="Comment",
                        placeholder="Enter your comment...",
                        lines=5
                    )
                    comment_btn = gr.Button("🔍 Check Comment", variant="primary")
                    
                    gr.Markdown("#### Try these examples:")
                    gr.Examples(
                        examples=[
                            ["Great article! Thanks for sharing."],
                            ["Nice post'; DROP TABLE users; --"],
                            ["I love this! ' OR '1'='1"],
                            ["Check out my website at example.com"],
                        ],
                        inputs=comment_input,
                        label="Example Comments"
                    )
                
                with gr.Column():
                    comment_risk = gr.Textbox(label="Risk Assessment", lines=1)
                    comment_confidence = gr.Textbox(label="Model Confidence", lines=1)
                    comment_suggestion = gr.Textbox(label="Security Suggestion", lines=2)
            
            comment_btn.click(
                fn=check_comment,
                inputs=comment_input,
                outputs=[comment_risk, comment_confidence, comment_suggestion]
            )
        
        # Tab 3: Custom Input
        with gr.Tab("⚡ Custom Input"):
            gr.Markdown("### Test any custom text or SQL query")
            
            with gr.Row():
                with gr.Column():
                    custom_input = gr.Textbox(
                        label="Custom Input",
                        placeholder="Enter any text, SQL query, or user input...",
                        lines=6
                    )
                    custom_btn = gr.Button("🔍 Analyze", variant="primary")
                    
                    gr.Markdown("#### Common SQL Injection Patterns:")
                    gr.Examples(
                        examples=[
                            ["SELECT * FROM users WHERE id = 1"],
                            ["SELECT * FROM users WHERE id = 1' OR '1'='1'--"],
                            ["1' UNION SELECT NULL, username, password FROM users--"],
                            ["'; DELETE FROM products WHERE '1'='1"],
                            ["admin' AND 1=0 UNION ALL SELECT NULL, NULL, NULL--"],
                        ],
                        inputs=custom_input,
                        label="SQL Injection Examples"
                    )
                
                with gr.Column():
                    custom_risk = gr.Textbox(label="Risk Assessment", lines=1)
                    custom_confidence = gr.Textbox(label="Model Confidence", lines=1)
                    custom_suggestion = gr.Textbox(label="Security Suggestion", lines=2)
            
            custom_btn.click(
                fn=check_comment,
                inputs=custom_input,
                outputs=[custom_risk, custom_confidence, custom_suggestion]
            )
    
    gr.Markdown("""
    ---
    ### 📚 About SQL Injection
    SQL injection is a code injection technique that attackers use to exploit vulnerabilities in applications.
    This model uses DistilBERT to identify potentially malicious patterns in user input.
    
    **Common Attack Patterns:**
    - `' OR '1'='1` - Always true condition
    - `'; DROP TABLE` - Database destruction
    - `' UNION SELECT` - Data extraction
    - `'--` or `'#` - Comment injection
    """)

# Launch interface
demo.launch(share=False, server_name="127.0.0.1", server_port=7860)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11104\979650565.py:42: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="SQL Injection Detection System") as demo:


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
